# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [22]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [23]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [24]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [25]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [26]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [27]:
# TO-DO: Create a Keyspace 
session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS sparkify 
                    WITH REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """)

#### Set Keyspace

In [28]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
session.execute("""
                   CREATE TABLE IF NOT EXISTS songplay_on_session 
                   (session_id int, item_in_session int, artist_name text, song_title text, song_length float, 
                   PRIMARY KEY(session_id, item_in_session))
                """)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay_on_session (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = """
            SELECT artist_name, song_title, song_length 
                FROM songplay_on_session 
            WHERE session_id = 338 
                AND item_in_session = 4
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Table 2 setup

In [11]:
session.execute("""
                   CREATE TABLE IF NOT EXISTS user_play_session 
                   (user_id int, session_id int, artist_name text, song_title text, item_in_session int, user_name text, 
                   PRIMARY KEY((user_id, session_id), item_in_session, song_title))
                """)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_play_session (user_id, session_id, artist_name, song_title, item_in_session, user_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), f'{line[1]} {line[4]}'))

In [13]:
query = """
            SELECT artist_name, song_title, user_name, item_in_session 
                FROM user_play_session
            WHERE user_id = 10 
                AND session_id= 182
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, row.song_title, row.user_name, row.item_in_session)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


#### Table 3 setup

In [45]:
## Query 3:
session.execute("""
                   CREATE TABLE IF NOT EXISTS song_listens 
                   (song_title text, user_name text, artist_name text, 
                   PRIMARY KEY((song_title), artist_name))
                """) 
# Setting the primary key with a clustering column to impede overwriting in case of same name songs. This asumes that a band does not have music with same name

In [46]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listens (song_title, user_name, artist_name) "
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], f'{line[1]} {line[4]}', line[0]))

In [52]:
query = """
            SELECT user_name
                FROM song_listens
            WHERE song_title = 'All Hands Against His Own'
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.user_name)

Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE songplay_on_session")

In [18]:
session.execute("DROP TABLE user_play_session")

In [43]:
session.execute("DROP TABLE song_listens")

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()

## The overall data modeling process

The objective of this notebook is to test and develop base skills with apache Cassandra. As a NOSQL database we have to model our data based on the queries that we want to do. Based on this, we made 3 tables based on 3 queries in this exercise

## Query 1: 
- Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

RESULT:
```
Faithless Music Matters (Mark Knight Dub) 495.30731201171875
```
For that a table with the session id as a Partition Key and item in session as a Clustering Column so that these can be used as the pivots in wich we retrieve our data. With this table we can retrieve any song, the artist that made and its duration. For that reason it was called songplay_on_session. The reason for the song duration being a float pointer is so that the source data does not be lost and that it does not interfers directly on this query. If it was a case of filtering by it could be rounded to its closest int number.


## Query 2:
- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

RESULT:  
```
Down To The Bone Keep On Keepin' On Sylvie Cruz 0  
Three Drives Greece 2000 Sylvie Cruz 1  
Sebastien Tellier Kilometer Sylvie Cruz 2  
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3  
```

This is a bit trickier data to model due the necessity of sorting in one of the columns. That was made using user_id and session_id as a composite Partition Key. For the case of song being sorted by item in session, the song column had to be put as a clustering column appearing after item in session. WIth this table you can retrieve any user session and their played song. For this reason this table was named: user_play_session.


## Query 3: 
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'  

RESULT: 
```
Sara Johnson  
```

This seems to be the simplest queries of them, but there is a catch that has to be addressed. If we use the song name as the only Partition Key there's the risk that songs with same name made by different artists overwrites each other. For that reason the artist name was added as a clustering column. With this query you can see the names of every users that played certain song!! For that reason it was named: song_listens.